In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

In [2]:
softmax_outputs = np.array([[0.7, 0.1, 0.2],
                            [0.1, 0.5, 0.4],
                            [0.02, 0.9, 0.08]])

In [3]:
class_targets = [0, 1, 1]

In [4]:
print(-np.log(softmax_outputs[[0, 1, 2], class_targets]))

[0.35667494 0.69314718 0.10536052]


In [5]:
nnfs.init()

In [6]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [7]:
class Activation_ReLU:
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [8]:
class Activation_Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probabilities

In [9]:
class loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss

In [10]:
class Loss_CategoricalCrossentropy(loss):
    def forward(self, y_pred, y_true):
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
            
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped*y_true, axis=1)
            
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [11]:
X, y = spiral_data(samples=100, classes=3)

In [12]:
dense1 = Layer_Dense(2, 3)
activation1 = Activation_ReLU()

In [13]:
dense2 = Layer_Dense(3, 3)
activation2 = Activation_Softmax()

In [14]:
dense1.forward(X)
activation1.forward(dense1.output)

In [15]:
dense2.forward(activation1.output)
activation2.forward(dense2.output)

In [16]:
print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.3333332  0.3333332  0.33333364]
 [0.3333329  0.33333293 0.3333342 ]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]


In [18]:
loss_function = Loss_CategoricalCrossentropy()
loss = loss_function.calculate(activation2.output, y)

In [20]:
print("Loss:", loss)

Loss: 1.0986104
